In [ ]:
# ==============================================
# FINE-TUNING GPT-2 SUR CNN/DAILYMAIL (5000 exemples)
# ==============================================

print("="*60)
print("FINE-TUNING GPT-2 (124M) - COMME L'ARTICLE")
print("="*60)

# ==============================================
# 1. INSTALLATIONS
# ==============================================

!pip install transformers datasets accelerate rouge-score -q

import torch
import numpy as np
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import gc
import os

# Nettoyage mémoire
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

print("✅ Bibliothèques installées")

# ==============================================
# 2. DATASET (5000 train, 1000 val, 1000 test)
# ==============================================

print("\n" + "="*60)
print("📊 CHARGEMENT DU DATASET")
print("="*60)

dataset = load_dataset("cnn_dailymail", "3.0.0")

# Split comme dans l'article
train_dataset = dataset["train"].select(range(5000))      # 5000 training
val_dataset = dataset["validation"].select(range(1000))   # 1000 validation
test_dataset = dataset["test"].select(range(1000))        # 1000 test

print(f"✅ Dataset prêt:")
print(f"  Training:   {len(train_dataset)} exemples")
print(f"  Validation: {len(val_dataset)} exemples")
print(f"  Test:       {len(test_dataset)} exemples")

# ==============================================
# 3. TOKENISATION GPT-2
# ==============================================

print("\n" + "="*60)
print("🔤 TOKENISATION GPT-2")
print("="*60)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Important pour GPT-2

def preprocess_function(examples):
    """Format pour GPT-2: [article] [separator] [summary]"""
    texts = []

    for article, highlight in zip(examples["article"], examples["highlights"]):
        # Format: article + séparateur + résumé
        text = f"ARTICLE: {article}\n\nSUMMARY: {highlight}"
        texts.append(text)

    # Tokeniser
    tokenized = tokenizer(
        texts,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors=None
    )

    # Pour GPT-2, les labels sont les mêmes que input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenisation en cours...")
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=train_dataset.column_names,
    desc="Tokenisation training"
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=val_dataset.column_names,
    desc="Tokenisation validation"
)

print("✅ Tokenisation terminée")

# ==============================================
# 4. MODÈLE GPT-2 (124M paramètres)
# ==============================================

print("\n" + "="*60)
print("🧠 CHARGEMENT DE GPT-2 (124M)")
print("="*60)

model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurer pour le padding
model.config.pad_token_id = tokenizer.pad_token_id

total_params = sum(p.numel() for p in model.parameters())
print(f"✅ GPT-2 chargé")
print(f"📊 Paramètres: {total_params/1e6:.1f}M")
print(f"📊 Device: {model.device}")

# ==============================================
# 5. CONFIGURATION DU FINE-TUNING
# ==============================================

print("\n" + "="*60)
print("⚙️  CONFIGURATION DU FINE-TUNING")
print("="*60)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-5000",
    overwrite_output_dir=True,
    num_train_epochs=5,  # Comme dans l'article
    per_device_train_batch_size=4,  # GPT-2 est plus léger
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Batch effectif = 16
    learning_rate=1e-5,  # Faible comme dans l'article
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs-gpt2",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    report_to="none",
    dataloader_pin_memory=False,
)

print("✅ Configuration définie:")
print(f"  • Epochs: 5 (comme l'article)")
print(f"  • Batch size: 4")
print(f"  • Learning rate: 1e-5")

# ==============================================
# 6. FINE-TUNING GPT-2
# ==============================================

print("\n" + "="*60)
print("🔥 DÉBUT DU FINE-TUNING GPT-2")
print("="*60)
print("⚠️  Cette étape prend 1-2 heures")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

try:
    train_result = trainer.train()
    print(f"\n✅ FINE-TUNING RÉUSSI !")
    print(f"⏱️  Temps: {train_result.metrics['train_runtime']/60:.1f} min")
    print(f"📉 Training loss: {train_result.metrics['train_loss']:.3f}")

except Exception as e:
    print(f"\n❌ ERREUR: {e}")
    print("\n🔄 Tentative avec batch_size=2...")

    # Réessayer avec batch size plus petit
    training_args.per_device_train_batch_size = 2
    training_args.per_device_eval_batch_size = 2
    training_args.gradient_accumulation_steps = 8

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
    )

    train_result = trainer.train()
    print(f"\n✅ FINE-TUNING RÉUSSI avec batch_size=2")

# ==============================================
# 7. SAUVEGARDE DU MODÈLE FINE-TUNÉ
# ==============================================

print("\n" + "="*60)
print("💾 SAUVEGARDE DU MODÈLE GPT-2 FINE-TUNÉ")
print("="*60)

model_save_path = "./gpt2_finetuned_5000"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"✅ Modèle GPT-2 fine-tuné sauvegardé dans: {model_save_path}")

# ==============================================
# 8. ÉVALUATION SUR TEST SET (1000 exemples)
# ==============================================

print("\n" + "="*60)
print("📊 ÉVALUATION ROUGE SUR TEST SET")
print("="*60)

!pip install rouge-score -q
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Fonction de génération pour GPT-2
def generate_summary_gpt2(text):
    """Génère un résumé avec GPT-2 fine-tuné - CORRIGÉ"""
    prompt = f"ARTICLE: {text[:800]}\n\nSUMMARY:"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],  # ⭐ AJOUTER
            max_new_tokens=100,  # ⭐ CORRECTION ICI (au lieu de max_length)
            min_new_tokens=30,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    full_output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    if "SUMMARY:" in full_output:
        return full_output.split("SUMMARY:")[-1].strip()
    return full_output

# Évaluation sur 1000 exemples
print(f"Évaluation sur 1000 exemples du test set...")

gpt2_rouge1 = []
gpt2_rouge2 = []
gpt2_rougeL = []
gpt2_rougeLsum = []

import time
start_time = time.time()

for i in range(1000):
    article = test_dataset[i]["article"]
    reference = test_dataset[i]["highlights"]

    generated = generate_summary_gpt2(article)
    scores = scorer.score(reference, generated)

    gpt2_rouge1.append(scores['rouge1'].fmeasure)
    gpt2_rouge2.append(scores['rouge2'].fmeasure)
    gpt2_rougeL.append(scores['rougeL'].fmeasure)
    gpt2_rougeLsum.append(scores['rougeLsum'].fmeasure)

    if (i + 1) % 100 == 0:
        progress = (i + 1) / 1000 * 100
        current_rouge1 = np.mean(gpt2_rouge1) * 100
        print(f"  {i+1}/1000 ({progress:.0f}%) - ROUGE-1: {current_rouge1:.1f}%")

eval_time = time.time() - start_time

# ==============================================
# 9. RÉSULTATS ROUGE (comme l'article)
# ==============================================

print("\n" + "="*60)
print("📈 RÉSULTATS ROUGE - GPT-2 FINE-TUNÉ")
print("="*60)

gpt2_r1 = np.mean(gpt2_rouge1) * 100
gpt2_r2 = np.mean(gpt2_rouge2) * 100
gpt2_rL = np.mean(gpt2_rougeL) * 100
gpt2_rLsum = np.mean(gpt2_rougeLsum) * 100

print(f"\n🎯 TES RÉSULTATS GPT-2 (1000 exemples):")
print(f"  ROUGE-1:    {gpt2_r1:.2f}%")
print(f"  ROUGE-2:    {gpt2_r2:.2f}%")
print(f"  ROUGE-L:    {gpt2_rL:.2f}%")
print(f"  ROUGE-Lsum: {gpt2_rLsum:.2f}%")

print(f"\n📊 STATISTIQUES:")
print(f"  Écart-type ROUGE-1: {np.std(gpt2_rouge1)*100:.2f}%")
print(f"  Temps d'évaluation: {eval_time/60:.1f} min")

# ==============================================
# 10. COMPARAISON AVEC L'ARTICLE
# ==============================================

print("\n" + "="*60)
print("📊 COMPARAISON AVEC L'ARTICLE (Table 3)")
print("="*60)

print(f"\n{'Modèle':<25} {'ROUGE-1':<10} {'ROUGE-2':<10} {'ROUGE-L':<10} {'ROUGE-Lsum':<10}")
print("-" * 65)
print(f"{'Article GPT-2':<25} {24.83:<10.2f} {16.92:<10.2f} {22.14:<10.2f} {21.07:<10.2f}")
print(f"{'Ton GPT-2 (5000 ex)':<25} {gpt2_r1:<10.2f} {gpt2_r2:<10.2f} {gpt2_rL:<10.2f} {gpt2_rLsum:<10.2f}")
print("-" * 65)

difference_rouge1 = gpt2_r1 - 24.83
print(f"\n📈 Différence ROUGE-1: {difference_rouge1:+.2f}%")

if difference_rouge1 > 0:
    print("✅ Ton modèle performe MIEUX que l'article !")
elif difference_rouge1 > -5:
    print("👍 Performance proche de l'article (normal avec moins de données)")
else:
    print("⚠️  Performance inférieure (normal: 5000 vs 287K exemples dans l'article)")

# ==============================================
# 11. SAUVEGARDE DES RÉSULTATS
# ==============================================

print("\n" + "="*60)
print("💾 SAUVEGARDE DES RÉSULTATS GPT-2")
print("="*60)

import json
from datetime import datetime

# Créer dossier résultats
results_dir = "./gpt2_finetuned_results"
os.makedirs(results_dir, exist_ok=True)

# Sauvegarder les résultats
results = {
    "model": "GPT-2 (124M) fine-tuned",
    "training": {
        "examples": 5000,
        "validation": 1000,
        "epochs": 5,
        "learning_rate": 1e-5,
        "batch_size": 4
    },
    "evaluation": {
        "test_examples": 1000,
        "rouge1": float(gpt2_r1),
        "rouge2": float(gpt2_r2),
        "rougeL": float(gpt2_rL),
        "rougeLsum": float(gpt2_rLsum),
        "std_rouge1": float(np.std(gpt2_rouge1) * 100)
    },
    "comparison_with_article": {
        "article_rouge1": 24.83,
        "article_rouge2": 16.92,
        "article_rougeL": 22.14,
        "article_rougeLsum": 21.07,
        "difference_rouge1": float(difference_rouge1)
    },
    "date": datetime.now().isoformat()
}

with open(os.path.join(results_dir, "results.json"), "w") as f:
    json.dump(results, f, indent=2)

print(f"✅ Résultats sauvegardés dans: {results_dir}/results.json")

# ==============================================
# 12. COMPARAISON GPT-2 vs BART
# ==============================================

print("\n" + "="*60)
print("📊 COMPARAISON GPT-2 vs BART")
print("="*60)

# Charger résultats BART si disponibles
bart_results_path = "./bart_finetuned_results/results.json"
if os.path.exists(bart_results_path):
    with open(bart_results_path, 'r') as f:
        bart_results = json.load(f)

    bart_rouge1 = bart_results["evaluation"]["rouge1"]

    print(f"\n🎯 COMPARAISON DES PERFORMANCES:")
    print(f"  • GPT-2 ROUGE-1: {gpt2_r1:.2f}%")
    print(f"  • BART ROUGE-1:  {bart_rouge1:.2f}%")
    print(f"  • Différence:    {bart_rouge1 - gpt2_r1:+.2f}%")

    if bart_rouge1 > gpt2_r1:
        print(f"\n✅ BART est supérieur à GPT-2 (confirmé)")
    else:
        print(f"\n⚠️  Résultat inattendu")
else:
    print("⚠️  Résultats BART non trouvés pour comparaison")

# ==============================================
# 13. TÉLÉCHARGEMENT
# ==============================================

print("\n" + "="*60)
print("📦 PRÉPARATION DU TÉLÉCHARGEMENT")
print("="*60)

import shutil

# Créer ZIP avec modèle + résultats
final_dir = "./gpt2_project_final"
os.makedirs(final_dir, exist_ok=True)

# Copier modèle
shutil.copytree(model_save_path, os.path.join(final_dir, "model"), dirs_exist_ok=True)
# Copier résultats
shutil.copy(os.path.join(results_dir, "results.json"), os.path.join(final_dir, "results.json"))

# Créer ZIP
zip_name = "gpt2_finetuned_project"
shutil.make_archive(zip_name, 'zip', final_dir)

# Télécharger
from google.colab import files
files.download(f"{zip_name}.zip")

print(f"\n✅ PROJET GPT-2 TERMINÉ !")
print(f"📦 Fichier: {zip_name}.zip")
print(f"📊 ROUGE-1: {gpt2_r1:.2f}%")
print(f"📈 Comparaison article: {difference_rouge1:+.2f}%")